# 数据队列 and 文件队列

### 数据队列

In [ ]:
import tensorflow as tf

#### 创建FIFOQueue先进先出队列，同时指定队列中可以保存的元素个数以及数据类型
#### 原型_init_(self,capacity,dtypes,shared_name,names,shapes,name)

In [ ]:
Queue = tf.FIFOQueue(2,'int32')

#### 使用FIFOQueue类的enqueue_many()函数初始化队列中的元素
#### 和初始化变量类似，在使用队列前需要明确调用初始化过程
#### #dequeue_many(self,n,name)

In [ ]:
queue_init = Queue.enqueue_many(([10,100],))

#### FIFOQueue类的dequeue()函数可以将队列队首的一个元素出队
#### #dequeue(self,name)

In [ ]:
a= Queue.dequeue()
b=a+10

#### FIFOQueue类的enqueue()函数可以将一个元素从队列队尾入队
#### enqueue(self,vals,name)

In [ ]:
Queue_en = Queue.enqueue([b])

In [ ]:
with tf.Session() as sess:
    queue_init.run()
    for i in range(5):
        v,_ =sess.run([a,Queue_en])
        print(v)

## 文件队列- 产生两个文件

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('./input/MNIST_data',dtype=tf.uint8,one_hot=True)

def _int64_feature(value):
    return tf.train.Feature(int64_list= tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

images = mnist.test.images
labels = mnist.test.labels
pixels = images.shape[1]
num_examples = mnist.test.num_examples

#### num_files定义总共写入多少个文件

In [ ]:
num_files = 2

for i in range(num_files):
    #将数据写入多个文件时，为区分这些文件可以添加后缀
    filename = ('./output/data_tfrecords-%.1d-of-%.1d' % (i,num_files))
    writer = tf.python_io.TFRecordWriter(filename)
    #将Example结构写入TFRecord文件。
    for index in range(num_examples):
        image_string = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(
            feature={
                    'pixels':_int64_feature(pixels),
                    'label': _int64_feature(np.argmax(labels[index])),
                    'image_raw':_bytes_feature(image_string)
                    
                }
            
            ))
        writer.write(example.SerializeToString())
    writer.close()
    

## 文件队列-- 队列文件读取

In [1]:
import tensorflow as tf

#### 使用match_filenames_once()函数获取符合正则表达式的所有文件
#### 函数原型match_filenames_once(pattern,name)

In [2]:
files = tf.train.match_filenames_once('./output/data_tfrecords-*') 

#### 通过string_input_producer()函数创建输入队列，输入队列的文件列表是files
#### 函数原型string_input_producer(string_tensor,num_epochs,shuffle,capacity,shared_name,name,cancel_op)
#### 函数的shuffle参数用于指定是否随即打乱读文件的顺序，在实际问题中会设置为True

In [3]:
filename_queue = tf.train.string_input_producer(files,shuffle=False)
reader = tf.TFRecordReader()
_,serialized_example = reader.read(filename_queue)

#### 解析读取的样例。

In [4]:
features = tf.parse_single_example(
        serialized_example,
        features={
        'image_raw': tf.FixedLenFeature([],tf.string),
        'pixels': tf.FixedLenFeature([],tf.int64),
        'label': tf.FixedLenFeature([],tf.int64)
    }
    )

In [5]:
images = tf.decode_raw(features['image_raw'],tf.uint8)
labels = tf.cast(features['label'],tf.int32)
pixels = tf.cast(features['pixels'],tf.int32)

In [10]:
with tf.Session() as sess:
    tf.local_variables_initializer().run()
    print(sess.run(files))
    coordinator = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coordinator)
    for i in range(6):
        print(sess.run([labels]))
    coordinator.request_stop()
    coordinator.join(threads)

[b'C:\\Users\\cgy\\Documents\\tensorflow book\\\xe8\xaf\xbb\xe5\x8f\x96\xe6\x95\xb0\xe6\x8d\xae\xe6\x96\xb9\xe6\xb3\x95\\output\\data_tfrecords-0-of-2'
 b'C:\\Users\\cgy\\Documents\\tensorflow book\\\xe8\xaf\xbb\xe5\x8f\x96\xe6\x95\xb0\xe6\x8d\xae\xe6\x96\xb9\xe6\xb3\x95\\output\\data_tfrecords-1-of-2']
[7]
[2]
[1]
[0]
[4]
[1]
